Revising the Senate file to cover both groups of legislators.

This is going to link together three datasets: Texas Legislature Online (TLO), Texas Election Commission (TEC),
and OpenStates.

Loading TLO first.

legislators_tlo.html was copied from the source of
http://www.legis.state.tx.us/Search/BillSearchLegislatorList.aspx?ID=usrLegislatorsFolder$cboAuthor&Leg=85

In [1]:
import pandas as pd
import re
import numpy as np

pattern = """\t<option value="(A[0-9]{3,5})">([^,]+)(, (.+))? \((H|S)-A[0-9]{3,5}\)</option>"""

fp = open('../data/inputs/legislators_tlo.html', encoding = 'utf-8')
lines = fp.readlines()

leg = []

for line in lines:
    # Regex applied to each line 
    match = re.search(pattern, line)
    legislator = [match.group(1), match.group(2), match.group(4), match.group(5)]
    if legislator[3] == "H":
        legislator[3] = "STATEREP"
    else: 
        legislator[3] = "STATESEN"
    leg.append(legislator)
        

In [2]:
# Cleaning text in TLO dataset

def substitute(string):
    if type(string) == str:
        string = string.replace("&#233;", "é")
        string = string.replace("&quot;", '"')
        string = string.replace("&#225;", 'á')
        string = string.replace("&#237;", "í")
        string = string.replace("&#241;", "ñ")
        string = string.replace("\t", "")
    return string

for line in leg:
    line[1] = substitute(line[1])
    line[2] = substitute(line[2])
    if line[1] == "Lucio":
        line[1] = "Lucio Jr."


In [3]:
# Turning TLO dataset into a dataframe

leg = pd.DataFrame(leg, columns = ["TLO_id","last_name","first_name", "filerHoldOfficeCd"])

In [4]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 200
leg

,TLO_id,last_name,first_name,filerHoldOfficeCd
0,A2100,Allen,None,STATEREP
1,A2125,Alonzo,None,STATEREP
2,A2135,Alvarado,None,STATEREP
3,A2150,Anchia,None,STATEREP
4,A2155,Anderson,"Charles ""Doc""",STATEREP
5,A2215,Anderson,Rodney,STATEREP
6,A3555,Arévalo,None,STATEREP
7,A2330,Ashby,None,STATEREP
8,A3200,Bailes,None,STATEREP
9,A2335,Bell,None,STATEREP


In [5]:
# splitting off nicknames and suffixes in the data from TLO

def parsenickname(first):
    nicknamePattern = '[\"\(](\w+)[\"\)]$'
    try:
        match = re.search(nicknamePattern, first)
        newFirst = first.split(match.group(0))[0].strip()
        return newFirst, match.group(1)
    except:
        return first, np.nan

"""   
def parsesuffix(name):
    suffixPattern = "(Jr\.|Sr\.|III)"
    if re.match(suffixPattern, name):
        match = re.search(suffixPattern, name)
        newName = name.strip(match.group(1))
        return newName.strip(), match.group(1)
    else:
        return name, np.nan
""" 

def parsesuffix(row):
    suffixPattern = "(Jr\.|Sr\.|III)"
    answers = [row["first_name"], row["last_name"], np.nan]
    if re.search(suffixPattern, row["last_name"]):
        match = re.search(suffixPattern, row["last_name"])
        newLast = row["last_name"].strip(match.group(1))
        answers = [row["first_name"], newLast.strip(), match.group(1)]
    elif type(row["first_name"]) == str and re.search(suffixPattern, row["first_name"]):
        match = re.search(suffixPattern, row["first_name"])
        newName = row["first_name"].strip(match.group(1))
        answers = [newName.strip(), row["last_name"], match.group(1)]
    if type(answers[0]) == str and len(answers[0]) == 0:
        answers[0] = np.nan
    return answers[0], answers[1], answers[2]

leg["first_name"], leg["filerNameShort"] = zip(*leg["first_name"].map(parsenickname))
leg["first_name"], leg["last_name"], leg["suffixes"] = zip(*leg.apply(parsesuffix, axis = 1))

leg

,TLO_id,last_name,first_name,filerHoldOfficeCd,filerNameShort,suffixes
0,A2100,Allen,None,STATEREP,NaN,NaN
1,A2125,Alonzo,None,STATEREP,NaN,NaN
2,A2135,Alvarado,None,STATEREP,NaN,NaN
3,A2150,Anchia,None,STATEREP,NaN,NaN
4,A2155,Anderson,Charles,STATEREP,Doc,NaN
5,A2215,Anderson,Rodney,STATEREP,NaN,NaN
6,A3555,Arévalo,None,STATEREP,NaN,NaN
7,A2330,Ashby,None,STATEREP,NaN,NaN
8,A3200,Bailes,None,STATEREP,NaN,NaN
9,A2335,Bell,None,STATEREP,NaN,NaN


In [6]:
# This loads a file from an Open States bulk download
# Not sure that {"district": object} is the right choice, but I wanted integers with NaN allowed.

openstates = pd.read_csv("../data/inputs/openstates/2017-06-02-tx-csv/tx_legislators.csv", index_col = "leg_id",
                        dtype = {"district": object})

openstates["first_name"], openstates["filerNameShort"] = zip(*openstates["first_name"].map(parsenickname))

In [7]:
# some of the openstates rows are easy to link because they include the TLO id as part of the URL 
# to the legislator's photo.

import re

def parseTLO(url):
    urlPattern = "small\/(A[0-9]{3,5})\.jpg$"
    try:
        match = re.search(urlPattern, url)
        return match.group(1)
    except:
        return np.nan
    
openstates["TLO_id"] = (openstates["photo_url"].map(parseTLO))

In [8]:
# Finding the senators whose photo file names didn't include their TLO id.

openstates[openstates["TLO_id"].isnull()][openstates["chamber"] == "upper"]

/Users/Matt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


,full_name,first_name,middle_name,last_name,suffixes,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at,filerNameShort,TLO_id
leg_id,,,,,,,,,,,,,,,,
TXL000198,"Eddie Lucio, Jr.",Eduardo A.,NaN,Lucio,Jr.,NaN,True,tx,upper,27,Democrat,http://www.house.state.tx.us/photos/members/36...,2010-06-19 03:51:42.083000,2017-06-01 10:38:26.283000,Eddie,NaN
TXL000312,José Menéndez,José,NaN,Menéndez,NaN,NaN,True,tx,upper,26,Democrat,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.467000,2017-06-01 10:38:26.641000,NaN,NaN
TXL000705,Dawn Buckingham,Dawn,NaN,Buckingham,NaN,NaN,True,tx,upper,24,Republican,NaN,2017-01-18 03:03:18.301000,2017-06-01 10:38:26.639000,NaN,NaN
TXL000707,Borris L. Miles,Borris L.,NaN,Miles,NaN,NaN,True,tx,upper,13,Democrat,NaN,2017-01-18 03:03:18.403000,2017-06-01 10:38:26.414000,NaN,NaN
TXL000716,Bryan Hughes,Bryan,NaN,Hughes,NaN,NaN,True,tx,upper,1,Republican,NaN,2017-01-27 05:42:08.204000,2017-06-01 10:38:26.497000,NaN,NaN


In [9]:
# just setting the remaining 5 senators' TLO_id directly.

tloID = {"TXL000198": "A1300", 
        "TXL000312": "A1110",
        "TXL000705": "A1125",
        "TXL000707": "A1115",
        "TXL000716": "A1135"}

for k in tloID.keys():
    openstates.set_value(k, "TLO_id", tloID[k])


In [10]:
openstates[openstates["chamber"] == "upper"]

,full_name,first_name,middle_name,last_name,suffixes,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at,filerNameShort,TLO_id
leg_id,,,,,,,,,,,,,,,,
TXL000190,Craig Estes,Craig,NaN,Estes,NaN,NaN,True,tx,upper,30,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.063000,2017-06-01 10:38:26.268000,NaN,A1180
TXL000195,Juan Hinojosa,Juan,NaN,Hinojosa,NaN,NaN,True,tx,upper,20,Democrat,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.076000,2017-06-01 10:38:26.808000,Chuy,A1250
TXL000196,Joan Huffman,Joan,NaN,Huffman,NaN,NaN,True,tx,upper,17,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.078000,2017-06-01 10:38:26.752000,NaN,A1260
TXL000198,"Eddie Lucio, Jr.",Eduardo A.,NaN,Lucio,Jr.,NaN,True,tx,upper,27,Democrat,http://www.house.state.tx.us/photos/members/36...,2010-06-19 03:51:42.083000,2017-06-01 10:38:26.283000,Eddie,A1300
TXL000199,Jane Nelson,Jane,NaN,Nelson,NaN,NaN,True,tx,upper,12,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.086000,2017-06-01 10:38:26.740000,NaN,A1450
TXL000200,Robert Nichols,Robert,NaN,Nichols,NaN,NaN,True,tx,upper,3,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.088000,2017-06-01 10:38:26.798000,NaN,A1400
TXL000203,Kel Seliger,Kel,NaN,Seliger,NaN,NaN,True,tx,upper,31,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.095000,2017-06-01 10:38:26.762000,NaN,A1460
TXL000206,Carlos Uresti,Carlos I.,NaN,Uresti,NaN,NaN,True,tx,upper,19,Democrat,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.103000,2017-06-01 10:38:26.767000,NaN,A1605
TXL000208,Kirk Watson,Kirk,NaN,Watson,NaN,NaN,True,tx,upper,14,Democrat,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.107000,2017-06-01 10:38:26.769000,NaN,A1610


In [11]:
# Trying to move Jr. to the suffix field when it occurs at the beginning of the first_name field.
# And not to overwrite suffix fields when there's no suffix in the name field.
# Probably need to use map command to take in the contents of both column and then write the contents of both.

# AFAIK, the only row affected by this is TXL000509

def parsefirstsuffix(row):
    suffixPattern = "^(Jr\.|Sr\.|III),"
    if re.match(suffixPattern, row['first_name']):
        match = re.search(suffixPattern, row['first_name'])
        newFirst = row['first_name'].split(match.group(0))[1]
        return newFirst, match.group(1)
    else:
        return row['first_name'], row['suffixes']

openstates["first_name"], openstates["suffixes"] = zip(*openstates.apply(parsefirstsuffix, axis=1))

In [12]:
# This will overwrite any existing middle name if the first name contains a space

def parsefirstmiddle(row):
    try: middle = row['middle_name']
    except: middle = np.nan
    if type(row['first_name']) == str and len(row['first_name'].split()) > 1:
        return row['first_name'].split()[0], row['first_name'].split()[1]
    else:
        return row['first_name'], middle

openstates["first_name"], openstates["middle_name"] = zip(*openstates.apply(parsefirstmiddle, axis=1))
leg["first_name"], leg["middle_name"] = zip(*leg.apply(parsefirstmiddle, axis=1))    

In [13]:
leg

,TLO_id,last_name,first_name,filerHoldOfficeCd,filerNameShort,suffixes,middle_name
0,A2100,Allen,None,STATEREP,NaN,NaN,NaN
1,A2125,Alonzo,None,STATEREP,NaN,NaN,NaN
2,A2135,Alvarado,None,STATEREP,NaN,NaN,NaN
3,A2150,Anchia,None,STATEREP,NaN,NaN,NaN
4,A2155,Anderson,Charles,STATEREP,Doc,NaN,NaN
5,A2215,Anderson,Rodney,STATEREP,NaN,NaN,NaN
6,A3555,Arévalo,None,STATEREP,NaN,NaN,NaN
7,A2330,Ashby,None,STATEREP,NaN,NaN,NaN
8,A3200,Bailes,None,STATEREP,NaN,NaN,NaN
9,A2335,Bell,None,STATEREP,NaN,NaN,NaN


In [14]:
# affects TXL000414

openstates["last_name"] = openstates["last_name"].apply(substitute)
openstates["full_name"] = openstates["full_name"].apply(substitute)

In [15]:
openSenate = openstates[openstates["chamber"] == "upper"]
legSenate = leg[leg["filerHoldOfficeCd"] == "STATESEN"]

In [16]:
senate = pd.merge(legSenate, openSenate.reset_index(), how='left', on='TLO_id')

In [17]:
openHouse = openstates[openstates["chamber"] == "lower"]
legHouse = leg[leg["filerHoldOfficeCd"] == "STATEREP"]

In [18]:
legHouse = legHouse.fillna(np.nan)

In [19]:
legLastOnly = leg[leg["first_name"].isnull()]

In [20]:
openLastOnly = pd.merge(legLastOnly, openHouse.reset_index(), how='left', on=['last_name'])

In [21]:
indexed_df = openLastOnly.set_index('TLO_id_x')

In [22]:
indexed_df[:5]

,last_name,first_name_x,filerHoldOfficeCd,filerNameShort_x,suffixes_x,middle_name_x,leg_id,full_name,first_name_y,middle_name_y,suffixes_y,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at,filerNameShort_y,TLO_id_y
TLO_id_x,,,,,,,,,,,,,,,,,,,,,,
A2100,Allen,None,STATEREP,NaN,NaN,NaN,TXL000214,Alma Allen,Alma,A.,NaN,NaN,True,tx,lower,131,Democratic,http://www.house.state.tx.us/photos/members/21...,2010-06-19 03:51:42.133000,2017-06-01 10:38:26.476000,NaN,NaN
A2125,Alonzo,None,STATEREP,NaN,NaN,NaN,TXL000215,Roberto Alonzo,Roberto,R.,NaN,NaN,True,tx,lower,104,Democratic,http://www.house.state.tx.us/photos/members/21...,2010-06-19 03:51:42.135000,2017-06-01 10:38:26.825000,NaN,NaN
A2135,Alvarado,None,STATEREP,NaN,NaN,NaN,TXL000216,Carol Alvarado,Carol,NaN,NaN,NaN,True,tx,lower,145,Democratic,http://www.house.state.tx.us/photos/members/21...,2010-06-19 03:51:42.137000,2017-06-01 10:38:25.930000,NaN,NaN
A2150,Anchia,None,STATEREP,NaN,NaN,NaN,TXL000217,Rafael Anchia,Rafael,NaN,NaN,NaN,True,tx,lower,103,Democratic,http://www.house.state.tx.us/photos/members/21...,2010-06-19 03:51:42.140000,2017-06-01 10:38:26.780000,NaN,NaN
A3555,Arévalo,None,STATEREP,NaN,NaN,NaN,TXL000683,"Arévalo, Diana",Diana,NaN,NaN,NaN,True,tx,lower,116,Democratic,http://www.house.state.tx.us/photos/members/35...,2017-01-07 05:53:03.970000,2017-06-01 10:38:26.361000,NaN,NaN


In [23]:
legHouse = legHouse.set_index("TLO_id")

In [24]:
legHouse["first_name"] = legHouse["first_name"].fillna(indexed_df["first_name_y"])

In [25]:
house = pd.merge(legHouse.reset_index(), openHouse.reset_index(), how='left', on=['first_name', 'last_name'])

In [26]:
senate[:1]

,TLO_id,last_name_x,first_name_x,filerHoldOfficeCd,filerNameShort_x,suffixes_x,middle_name_x,leg_id,full_name,first_name_y,middle_name_y,last_name_y,suffixes_y,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at,filerNameShort_y
0,A1055,Bettencourt,None,STATESEN,NaN,NaN,NaN,TXL000482,Paul Bettencourt,Paul,NaN,Bettencourt,NaN,NaN,True,tx,upper,7,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2015-01-14 00:53:27.393000,2017-06-01 10:38:26.745000,NaN


In [27]:
senate = senate.rename(index=str, columns={"last_name_x": "last_name", "first_name_y": "first_name"})
senate = senate.drop(['first_name_x', 'last_name_y'], axis=1)

In [28]:
house[:1]

,TLO_id_x,last_name,first_name,filerHoldOfficeCd,filerNameShort_x,suffixes_x,middle_name_x,leg_id,full_name,middle_name_y,suffixes_y,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at,filerNameShort_y,TLO_id_y
0,A2100,Allen,Alma,STATEREP,NaN,NaN,NaN,TXL000214,Alma Allen,A.,NaN,NaN,True,tx,lower,131,Democratic,http://www.house.state.tx.us/photos/members/21...,2010-06-19 03:51:42.133000,2017-06-01 10:38:26.476000,NaN,NaN


In [29]:
house = house.rename(index=str, columns={"TLO_id_x": "TLO_id"})
house = house.drop(['TLO_id_y'], axis=1)

In [30]:
# openTLOouter = pd.merge(legHouse.reset_index(), openHouse.reset_index(), how='outer', on=['first_name', 'last_name'])

In [31]:
senate

,TLO_id,last_name,filerHoldOfficeCd,filerNameShort_x,suffixes_x,middle_name_x,leg_id,full_name,first_name,middle_name_y,suffixes_y,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at,filerNameShort_y
0,A1055,Bettencourt,STATESEN,NaN,NaN,NaN,TXL000482,Paul Bettencourt,Paul,NaN,NaN,NaN,True,tx,upper,7,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2015-01-14 00:53:27.393000,2017-06-01 10:38:26.745000,NaN
1,A1080,Birdwell,STATESEN,NaN,NaN,NaN,TXL000364,Brian Birdwell,Brian,NaN,NaN,NaN,True,tx,upper,22,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-07-17 08:38:47.948000,2017-06-01 10:38:26.747000,NaN
2,A1125,Buckingham,STATESEN,NaN,NaN,NaN,TXL000705,Dawn Buckingham,Dawn,NaN,NaN,NaN,True,tx,upper,24,Republican,NaN,2017-01-18 03:03:18.301000,2017-06-01 10:38:26.639000,NaN
3,A1090,Burton,STATESEN,NaN,NaN,NaN,TXL000487,Konni Burton,Konni,NaN,NaN,NaN,True,tx,upper,10,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2015-01-14 00:53:27.536000,2017-06-01 10:38:26.492000,NaN
4,A1010,Campbell,STATESEN,NaN,NaN,NaN,TXL000419,Donna Campbell,Donna,NaN,NaN,NaN,True,tx,upper,25,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2013-01-09 00:29:35.053000,2017-06-01 10:38:26.288000,NaN
5,A1040,Creighton,STATESEN,NaN,NaN,NaN,TXL000240,Brandon Creighton,Brandon,NaN,NaN,NaN,True,tx,upper,4,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.210000,2017-06-01 10:38:26.803000,NaN
6,A1180,Estes,STATESEN,NaN,NaN,NaN,TXL000190,Craig Estes,Craig,NaN,NaN,NaN,True,tx,upper,30,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.063000,2017-06-01 10:38:26.268000,NaN
7,A1035,Garcia,STATESEN,NaN,NaN,NaN,TXL000473,Sylvia R. Garcia,Sylvia,R.,NaN,NaN,True,tx,upper,6,Democrat,http://www.legdir.legis.state.tx.us/FlashCardD...,2013-03-12 18:18:45.435000,2017-06-01 10:35:57.613000,NaN
8,A1050,Hall,STATESEN,NaN,NaN,NaN,TXL000503,Bob Hall,Bob,NaN,NaN,NaN,True,tx,upper,2,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2015-01-14 00:53:27.965000,2017-06-01 10:38:26.801000,NaN
9,A1015,Hancock,STATESEN,NaN,NaN,NaN,TXL000271,Kelly Hancock,Kelly,NaN,NaN,NaN,True,tx,upper,9,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.331000,2017-06-01 10:38:26.805000,NaN


In [32]:
# senate = tloOpen[tloOpen["ctaSeekOfficeCd"] == "STATESEN"]

In [33]:
# This loads a file from the Texas Election Commission

tx = pd.read_csv("../data/inputs/TEC_CF_CSV/filers.csv", dtype = {"filerIdent": int}, 
                 parse_dates = ['filerEffStartDt','filerEffStopDt', 'treasEffStartDt', 'treasEffStopDt'])

/Users/Matt/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (9,38,43,50,54,79,80,109,112,131) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
tx = tx[tx["filerTypeCd"] == 'COH']
tx = tx.rename(columns={"filerHoldOfficeDistrict": "district"})

In [35]:
# removed 'STATEREP',  to focus on Senate
# removed [tx['ctaSeekOfficeCd'].isin(['STATESEN'], changed to filerHoldOfficeCd 

txSenate = tx[tx['filerHoldOfficeCd'].isin(['STATESEN'])][tx['filerFilerpersStatusCd'] != "NOT_OFFICEHOLDER"]
txHouse = tx[tx['filerHoldOfficeCd'].isin(['STATEREP'])][tx['filerFilerpersStatusCd'] != "NOT_OFFICEHOLDER"]

/Users/Matt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/Users/Matt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [36]:
txSenate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31 entries, 40 to 13012
Columns: 132 entries, recordType to chairPrimaryPhoneExt
dtypes: datetime64[ns](4), float64(24), int64(1), object(103)
memory usage: 32.2+ KB


In [37]:
txHouse.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 1757 to 14069
Columns: 132 entries, recordType to chairPrimaryPhoneExt
dtypes: datetime64[ns](4), float64(24), int64(1), object(103)
memory usage: 155.9+ KB


In [38]:
senateAll = pd.merge(senate, txSenate, how='left', on = 'district')
houseAll = pd.merge(house, txHouse, how='left', on = 'district')

In [39]:
houseAll[:1]

,TLO_id,last_name,first_name,filerHoldOfficeCd_x,filerNameShort_x,suffixes_x,middle_name_x,leg_id,full_name,middle_name_y,suffixes_y,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at,filerNameShort_y,recordType,filerIdent,filerTypeCd,filerName,unexpendContribFilerFlag,modifiedElectCycleFlag,filerJdiCd,committeeStatusCd,ctaSeekOfficeCd,ctaSeekOfficeDistrict,ctaSeekOfficePlace,ctaSeekOfficeDescr,ctaSeekOfficeCountyCd,ctaSeekOfficeCountyDescr,filerPersentTypeCd,filerNameOrganization,filerNameLast,filerNameSuffixCd,filerNameFirst,filerNamePrefixCd,filerNameShort,filerStreetAddr1,filerStreetAddr2,filerStreetCity,filerStreetStateCd,filerStreetCountyCd,filerStreetCountryCd,filerStreetPostalCode,filerStreetRegion,filerMailingAddr1,filerMailingAddr2,filerMailingCity,filerMailingStateCd,filerMailingCountyCd,filerMailingCountryCd,filerMailingPostalCode,filerMailingRegion,filerPrimaryUsaPhoneFlag,filerPrimaryPhoneNumber,filerPrimaryPhoneExt,filerHoldOfficeCd_y,filerHoldOfficePlace,filerHoldOfficeDescr,filerHoldOfficeCountyCd,filerHoldOfficeCountyDescr,filerFilerpersStatusCd,filerEffStartDt,filerEffStopDt,contestSeekOfficeCd,contestSeekOfficeDistrict,contestSeekOfficePlace,contestSeekOfficeDescr,contestSeekOfficeCountyCd,contestSeekOfficeCountyDescr,treasPersentTypeCd,treasNameOrganization,treasNameLast,treasNameSuffixCd,treasNameFirst,treasNamePrefixCd,treasNameShort,treasStreetAddr1,treasStreetAddr2,treasStreetCity,treasStreetStateCd,treasStreetCountyCd,treasStreetCountryCd,treasStreetPostalCode,treasStreetRegion,treasMailingAddr1,treasMailingAddr2,treasMailingCity,treasMailingStateCd,treasMailingCountyCd,treasMailingCountryCd,treasMailingPostalCode,treasMailingRegion,treasPrimaryUsaPhoneFlag,treasPrimaryPhoneNumber,treasPrimaryPhoneExt,treasAppointorNameLast,treasAppointorNameFirst,treasFilerpersStatusCd,treasEffStartDt,treasEffStopDt,assttreasPersentTypeCd,assttreasNameOrganization,assttreasNameLast,assttreasNameSuffixCd,assttreasNameFirst,assttreasNamePrefixCd,assttreasNameShort,assttreasStreetAddr1,assttreasStreetAddr2,assttreasStreetCity,assttreasStreetStateCd,assttreasStreetCountyCd,assttreasStreetCountryCd,assttreasStreetPostalCode,assttreasStreetRegion,assttreasPrimaryUsaPhoneFlag,assttreasPrimaryPhoneNumber,assttreasPrimaryPhoneExt,assttreasAppointorNameLast,assttreasAppointorNameFirst,chairPersentTypeCd,chairNameOrganization,chairNameLast,chairNameSuffixCd,chairNameFirst,chairNamePrefixCd,chairNameShort,chairStreetAddr1,chairStreetAddr2,chairStreetCity,chairStreetStateCd,chairStreetCountyCd,chairStreetCountryCd,chairStreetPostalCode,chairStreetRegion,chairMailingAddr1,chairMailingAddr2,chairMailingCity,chairMailingStateCd,chairMailingCountyCd,chairMailingCountryCd,chairMailingPostalCode,chairMailingRegion,chairPrimaryUsaPhoneFlag,chairPrimaryPhoneNumber,chairPrimaryPhoneExt
0,A2100,Allen,Alma,STATEREP,NaN,NaN,NaN,TXL000214,Alma Allen,A.,NaN,NaN,True,tx,lower,131,Democratic,http://www.house.state.tx.us/photos/members/21...,2010-06-19 03:51:42.133000,2017-06-01 10:38:26.476000,NaN,FILER,19673,COH,"Allen, Alma A. (The Honorable)",N,N,NaN,NaN,STATEREP,131,NaN,NaN,NaN,NaN,INDIVIDUAL,NaN,Allen,NaN,Alma A.,HONORABLE,NaN,"3401 Louisiana St., Ste. 250",NaN,Houston,TX,NaN,USA,77002-9546,NaN,10709 Marsha Ln.,NaN,Houston,TX,NaN,USA,77024,NaN,Y,2816423426,NaN,STATEREP,NaN,NaN,NaN,NaN,CURRENT_OFFICEHOLDER,2005-01-01,NaT,STATEREP,131,NaN,NaN,NaN,NaN,INDIVIDUAL,NaN,Jackson,NaN,Alfred,MR,NaN,10709 Marsha Ln.,NaN,Houston,TX,NaN,USA,77024,NaN,"7670 Woodway, Ste. 110",NaN,Houston,TX,NaN,USA,77063,NaN,Y,7.13908e+09,NaN,NaN,NaN,CURRENT_OFFICEHOLDER,2005-01-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
houseAll[["last_name", "full_name", "filerName", "filerIdent"]][:5]

,last_name,full_name,filerName,filerIdent
0,Allen,Alma Allen,"Allen, Alma A. (The Honorable)",19673
1,Alonzo,Roberto Alonzo,"Alonzo, Roberto R. (The Honorable)",19681
2,Alvarado,Carol Alvarado,"Alvarado, Carol (The Honorable)",24376
3,Anchia,Rafael Anchia,"Anchia, Rafael M. (The Honorable)",54808
4,Anderson,Charles Anderson,"Anderson, Charles (The Honorable)",51449


In [41]:
senateAll[["last_name", "full_name", "filerName", "filerIdent"]][:5]

,last_name,full_name,filerName,filerIdent
0,Bettencourt,Paul Bettencourt,"Bettencourt, Paul (The Honorable)",69337.0
1,Birdwell,Brian Birdwell,"Birdwell, Brian D. (The Honorable)",62137.0
2,Buckingham,Dawn Buckingham,"Buckingham, Dawn C. (The Honorable)",69001.0
3,Burton,Konni Burton,"Burton, Konni L. (The Honorable)",69199.0
4,Campbell,Donna Campbell,"Campbell M.D., Donna (The Honorable)",67809.0


In [42]:
senateAll["filerIdent"] = senateAll["filerIdent"].astype(object)

In [47]:
senateAll[:1]

,last_name,filerHoldOfficeCd_x,filerNameShort_x,suffixes_x,middle_name_x,leg_id,full_name,first_name,middle_name_y,suffixes_y,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at,filerNameShort_y,recordType,filerIdent,filerTypeCd,filerName,unexpendContribFilerFlag,modifiedElectCycleFlag,filerJdiCd,committeeStatusCd,ctaSeekOfficeCd,ctaSeekOfficeDistrict,ctaSeekOfficePlace,ctaSeekOfficeDescr,ctaSeekOfficeCountyCd,ctaSeekOfficeCountyDescr,filerPersentTypeCd,filerNameOrganization,filerNameLast,filerNameSuffixCd,filerNameFirst,filerNamePrefixCd,filerNameShort,filerStreetAddr1,filerStreetAddr2,filerStreetCity,filerStreetStateCd,filerStreetCountyCd,filerStreetCountryCd,filerStreetPostalCode,filerStreetRegion,filerMailingAddr1,filerMailingAddr2,filerMailingCity,filerMailingStateCd,filerMailingCountyCd,filerMailingCountryCd,filerMailingPostalCode,filerMailingRegion,filerPrimaryUsaPhoneFlag,filerPrimaryPhoneNumber,filerPrimaryPhoneExt,filerHoldOfficeCd_y,filerHoldOfficePlace,filerHoldOfficeDescr,filerHoldOfficeCountyCd,filerHoldOfficeCountyDescr,filerFilerpersStatusCd,filerEffStartDt,filerEffStopDt,contestSeekOfficeCd,contestSeekOfficeDistrict,contestSeekOfficePlace,contestSeekOfficeDescr,contestSeekOfficeCountyCd,contestSeekOfficeCountyDescr,treasPersentTypeCd,treasNameOrganization,treasNameLast,treasNameSuffixCd,treasNameFirst,treasNamePrefixCd,treasNameShort,treasStreetAddr1,treasStreetAddr2,treasStreetCity,treasStreetStateCd,treasStreetCountyCd,treasStreetCountryCd,treasStreetPostalCode,treasStreetRegion,treasMailingAddr1,treasMailingAddr2,treasMailingCity,treasMailingStateCd,treasMailingCountyCd,treasMailingCountryCd,treasMailingPostalCode,treasMailingRegion,treasPrimaryUsaPhoneFlag,treasPrimaryPhoneNumber,treasPrimaryPhoneExt,treasAppointorNameLast,treasAppointorNameFirst,treasFilerpersStatusCd,treasEffStartDt,treasEffStopDt,assttreasPersentTypeCd,assttreasNameOrganization,assttreasNameLast,assttreasNameSuffixCd,assttreasNameFirst,assttreasNamePrefixCd,assttreasNameShort,assttreasStreetAddr1,assttreasStreetAddr2,assttreasStreetCity,assttreasStreetStateCd,assttreasStreetCountyCd,assttreasStreetCountryCd,assttreasStreetPostalCode,assttreasStreetRegion,assttreasPrimaryUsaPhoneFlag,assttreasPrimaryPhoneNumber,assttreasPrimaryPhoneExt,assttreasAppointorNameLast,assttreasAppointorNameFirst,chairPersentTypeCd,chairNameOrganization,chairNameLast,chairNameSuffixCd,chairNameFirst,chairNamePrefixCd,chairNameShort,chairStreetAddr1,chairStreetAddr2,chairStreetCity,chairStreetStateCd,chairStreetCountyCd,chairStreetCountryCd,chairStreetPostalCode,chairStreetRegion,chairMailingAddr1,chairMailingAddr2,chairMailingCity,chairMailingStateCd,chairMailingCountyCd,chairMailingCountryCd,chairMailingPostalCode,chairMailingRegion,chairPrimaryUsaPhoneFlag,chairPrimaryPhoneNumber,chairPrimaryPhoneExt
TLO_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A1055,Bettencourt,STATESEN,NaN,NaN,NaN,TXL000482,Paul Bettencourt,Paul,NaN,NaN,NaN,True,tx,upper,7,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2015-01-14 00:53:27.393000,2017-06-01 10:38:26.745000,NaN,FILER,69337,COH,"Bettencourt, Paul (The Honorable)",N,N,NaN,NaN,STATESEN,7,NaN,NaN,NaN,NaN,INDIVIDUAL,NaN,Bettencourt,NaN,Paul,HONORABLE,NaN,"1 E. Greenway Plz., Ste. 225",NaN,Houston,TX,NaN,USA,77046,NaN,"1 E. Greenway Plz., Ste. 225",NaN,Houston,TX,NaN,USA,77046,NaN,Y,7.13526e+09,NaN,STATESEN,NaN,NaN,NaN,NaN,CURRENT_OFFICEHOLDER,2015-01-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,INDIVIDUAL,NaN,Boylan,NaN,Michael,MR,NaN,"1 E. Greenway Plz., Ste. 225",NaN,Houston,TX,NaN,USA,77046,NaN,"1 E. Greenway Plz., Ste. 225",NaN,Houston,TX,NaN,USA,77046,NaN,Y,7.13526e+09,NaN,NaN,NaN,CURRENT_OFFICEHOLDER,2015-01-01,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [44]:
senateAll = senateAll.set_index('TLO_id')
houseAll = houseAll.set_index('TLO_id')

In [45]:
# These have a lot of duplicative columns that will only be useful for error checking.

senateAll.to_csv("../data/senateCrosswalk.csv")
houseAll.to_csv("../data/houseCrosswalk.csv")

In [48]:
# These extra redundant files have just the ID numbers and nothing else.

senateAll[["leg_id", "filerIdent"]].to_csv("../data/senateIDs.csv")
houseAll[["leg_id", "filerIdent"]].to_csv("../data/houseIDs.csv")